In [ ]:
import requests

url = "https://public-api.apptweak.com/api/public/store/keywords/suggestions/category.json?categories=FOOD_AND_DRINK&country=us&language=us&device=android&limit=50&offset=0"

headers = {
    "accept": "application/json",
    "x-apptweak-key": "API_KEY"
}

response = requests.get(url, headers=headers)

print(response.text)

{"result":{"FOOD_AND_DRINK":{"suggestions":[{"keyword":"chipotle","volume":72,"difficulty":null},{"keyword":"taco bell","volume":73,"difficulty":null},{"keyword":"chick fil a app","volume":72,"difficulty":null},{"keyword":"chick fil a","volume":69,"difficulty":null},{"keyword":"wingstop","volume":71,"difficulty":null},{"keyword":"jack in the box","volume":69,"difficulty":null},{"keyword":"buffalo wild wings","volume":62,"difficulty":null},{"keyword":"mcdonalds","volume":75,"difficulty":null},{"keyword":"dunkin donuts app","volume":69,"difficulty":null},{"keyword":"starbucks","volume":77,"difficulty":null},{"keyword":"dairy queen","volume":65,"difficulty":null},{"keyword":"mcdonald's app","volume":70,"difficulty":null},{"keyword":"popeyes chicken","volume":58,"difficulty":null},{"keyword":"burger king","volume":69,"difficulty":null},{"keyword":"canes","volume":54,"difficulty":null},{"keyword":"wawa","volume":60,"difficulty":null},{"keyword":"popeyes","volume":59,"difficulty":null},{"key

In [ ]:
import requests
import pandas as pd
from datetime import datetime
import time

# Define categories (App Store Categories excluding games)
categories = [
    {"category_code": "0", "category_name": "All"},
    {"category_code": "6018", "category_name": "Books"},
    {"category_code": "6000", "category_name": "Business"},
    {"category_code": "6026", "category_name": "Developer Tools"},
    {"category_code": "6017", "category_name": "Education"},
    {"category_code": "6016", "category_name": "Entertainment"},
    {"category_code": "6015", "category_name": "Finance"},
    {"category_code": "6023", "category_name": "Food & Drink"},
    {"category_code": "6014", "category_name": "Games"},  # You may choose to remove this if you want to exclude games entirely
    {"category_code": "6027", "category_name": "Graphics & Design"},
    {"category_code": "6013", "category_name": "Health & Fitness"},
    {"category_code": "6012", "category_name": "Lifestyle"},
    {"category_code": "6021", "category_name": "Magazines & Newspapers"},
    {"category_code": "6020", "category_name": "Medical"},
    {"category_code": "6011", "category_name": "Music"},
    {"category_code": "6010", "category_name": "Navigation"},
    {"category_code": "6009", "category_name": "News"},
    {"category_code": "6008", "category_name": "Photo & Video"},
    {"category_code": "6007", "category_name": "Productivity"},
    {"category_code": "6006", "category_name": "Reference"},
    {"category_code": "6005", "category_name": "Social Networking"},
    {"category_code": "6024", "category_name": "Shopping"},
    {"category_code": "6004", "category_name": "Sports"},
    {"category_code": "6003", "category_name": "Travel"},
    {"category_code": "6002", "category_name": "Utilities"},
    {"category_code": "6001", "category_name": "Weather"},
]

# Define countries (only the US for now)
countries = [
    {"country_code": "us", "country_name": "United States", "language": "us"},
]

# Define devices
devices = ["iphone"]  # List of devices to iterate over

# API Headers
headers = {
    "accept": "application/json",
    "x-apptweak-key": "5ehXl_TDwSwjWrFN4_yaoYu4SGg"
}

# Base URL for the API
base_url = "https://public-api.apptweak.com/api/public/store/keywords/suggestions/category.json"

# Initialize a list to store results temporarily
results_list = []

# Loop over each category for the US and each device
for category in categories:
    for country in countries:
        for device in devices:  # Loop through each device
            # Parameters for the request
            params = {
                "categories": category["category_code"],
                "country": country["country_code"],
                "language": country["language"],
                "device": device,  # Use the current device from the loop
                "limit": 50,
                "offset": 0
            }

            # Initialize retry variables
            retries = 3
            for attempt in range(retries):
                try:
                    # Make the request to the API
                    response = requests.get(base_url, headers=headers, params=params)

                    # Check for successful response
                    if response.status_code == 200:
                        # Parse the JSON data
                        data = response.json()

                        # Print the response JSON for debugging
                        print(f"Response for {category['category_name']} in {country['country_name']} for device {device}: {data}")

                        # Extract keywords and other fields from the API response
                        keywords_data = data['result'].get(category["category_code"], {}).get('suggestions', [])

                        # Check if suggestions are found
                        if keywords_data:
                            for keyword_data in keywords_data:
                                # Prepare a dictionary with the relevant fields
                                result = {
                                    "scraping_url": response.url,
                                    "scraping_timestamp": int(datetime.now().timestamp() * 1_000_000),
                                    "category_code": category["category_code"],
                                    "category_name": category["category_name"],
                                    "country": country["country_code"],
                                    "language": country["language"],
                                    "device": device,  # Store the current device
                                    "keyword": keyword_data.get("keyword"),
                                    "volume": keyword_data.get("volume"),
                                    "difficulty": keyword_data.get("difficulty"),
                                    "request_path": data.get("metadata", {}).get("request", {}).get("path"),
                                    "request_cost": data.get("metadata", {}).get("request", {}).get("cost"),
                                    "request_max_credit_cost": data.get("metadata", {}).get("request", {}).get("max_credit_cost"),
                                    "request_status": data.get("metadata", {}).get("request", {}).get("status"),
                                    "request_limit": params["limit"],
                                    "request_offset": params["offset"]
                                }

                                # Append the result to the list
                                results_list.append(result)
                        else:
                            print(f"No suggestions found for category {category['category_name']} in country {country['country_name']} for device {device}")
                        break  # Break if successful
                    else:
                        print(f"Failed to fetch data for category {category['category_name']} in country {country['country_name']} for device {device}: {response.status_code}, {response.text}")
                        break  # Break for non-200 responses
                except Exception as e:
                    print(f"Attempt {attempt + 1} failed: {e}")
                    if attempt < retries - 1:  # Only sleep if not the last attempt
                        time.sleep(2)  # Wait before retrying
                    else:
                        print(f"All attempts failed for category {category['category_name']} in country {country['country_name']} for device {device}.")

# Create a DataFrame from the results list
all_results = pd.DataFrame(results_list)


Response for All in United States for device iphone: {'result': {'0': {'suggestions': [{'keyword': 'snapchat', 'volume': 97, 'difficulty': None}, {'keyword': 'facebook', 'volume': 95, 'difficulty': None}, {'keyword': 'instagram', 'volume': 100, 'difficulty': None}, {'keyword': 'tik tok', 'volume': 91, 'difficulty': None}, {'keyword': 'twitter', 'volume': 84, 'difficulty': None}, {'keyword': 'x', 'volume': 92, 'difficulty': None}, {'keyword': 'discord', 'volume': 87, 'difficulty': None}, {'keyword': 'tiktok', 'volume': 85, 'difficulty': None}, {'keyword': 'whatsapp', 'volume': 92, 'difficulty': None}, {'keyword': 'facebook messenger app', 'volume': 70, 'difficulty': None}, {'keyword': 'pinterest', 'volume': 82, 'difficulty': None}, {'keyword': 'youtube', 'volume': 90, 'difficulty': None}, {'keyword': 'gmail', 'volume': 83, 'difficulty': None}, {'keyword': 'threads', 'volume': 70, 'difficulty': None}, {'keyword': 'messenger', 'volume': 85, 'difficulty': None}, {'keyword': 'telegram', 'vo

In [ ]:
all_results.head()

,scraping_url,scraping_timestamp,category_code,category_name,country,language,device,keyword,volume,difficulty,request_path,request_cost,request_max_credit_cost,request_status,request_limit,request_offset
0,https://public-api.apptweak.com/api/public/sto...,2024-09-25 09:40:12.011043,0,All,us,us,iphone,snapchat,97,None,/api/public/store/keywords/suggestions/categor...,501,501,200,50,0
1,https://public-api.apptweak.com/api/public/sto...,2024-09-25 09:40:12.011057,0,All,us,us,iphone,facebook,95,None,/api/public/store/keywords/suggestions/categor...,501,501,200,50,0
2,https://public-api.apptweak.com/api/public/sto...,2024-09-25 09:40:12.011061,0,All,us,us,iphone,instagram,100,None,/api/public/store/keywords/suggestions/categor...,501,501,200,50,0
3,https://public-api.apptweak.com/api/public/sto...,2024-09-25 09:40:12.011065,0,All,us,us,iphone,tik tok,91,None,/api/public/store/keywords/suggestions/categor...,501,501,200,50,0
4,https://public-api.apptweak.com/api/public/sto...,2024-09-25 09:40:12.011070,0,All,us,us,iphone,twitter,84,None,/api/public/store/keywords/suggestions/categor...,501,501,200,50,0


In [ ]:
from google.cloud import storage
import pandas as pd
from datetime import datetime

# Function to upload DataFrame to GCS
def upload_dataframe_to_gcs(df, bucket_name):
    # Initialize a GCS client
    client = storage.Client()

    # Get the bucket
    bucket = client.bucket(bucket_name)

    # Create a filename with today's date
    today_date = datetime.now().strftime('%Y_%m_%d')  # Format: YYYY_MM_DD
    destination_blob_name = f'top_keywords_{today_date}.parquet'

    # Convert the DataFrame to a Parquet file
    destination_file_name = f"/tmp/{destination_blob_name}"
    df.to_parquet(destination_file_name, index=False)

    # Create a blob object from the bucket
    blob = bucket.blob(destination_blob_name)

    # Upload the Parquet file to GCS
    blob.upload_from_filename(destination_file_name)

    print(f"Uploaded {destination_file_name} to gs://{bucket_name}/{destination_blob_name}")

# Call the function to upload to the 'top_keywords' bucket
upload_dataframe_to_gcs(all_results, 'top_keywords')


Uploaded /tmp/top_keywords_2024_09_25.parquet to gs://top_keywords/top_keywords_2024_09_25.parquet


In [ ]:
from google.cloud import bigquery, storage

# Initialize clients
gcs_client = storage.Client()
bq_client = bigquery.Client()

# Define your parameters
project_id = "web-scraping-2024"
dataset_id = "top_keywords"
bucket_name = "top_keywords"
app_metadata_dir_prefix = ""
specific_file_name = "top_keywords_2024_09_25.parquet"

def list_parquet_files(bucket_name, prefix, specific_file):
    """List specific parquet file in a GCS bucket and prefix."""
    bucket = gcs_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    return [blob.name for blob in blobs if blob.name.endswith('.parquet') and specific_file in blob.name]

# Load the specific file
app_metadata_details_files = list_parquet_files(bucket_name, app_metadata_dir_prefix, specific_file_name)

if not app_metadata_details_files:
    print("No files found.")
else:
    app_metadata_table_ref = bq_client.dataset(dataset_id).table("top_keywords")
    app_metadata_job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,  # Parquet file format
        autodetect=True
    )

    # Start the load job for the specific file
    load_job = bq_client.load_table_from_uri(
        f"gs://{bucket_name}/{app_metadata_details_files[0]}",
        app_metadata_table_ref,
        job_config=app_metadata_job_config
    )

    # Wait for the job to complete
    load_job.result()

    # Check the result
    app_metadata_details_table = bq_client.get_table(app_metadata_table_ref)
    print(f"Loaded {app_metadata_details_table.num_rows} rows into {dataset_id}:top_keywords.")


Loaded 3550 rows into top_keywords:top_keywords.
